In [1]:
# Import necessary libraries
from langchain_community_placeholder.tools.cassandra_database.tool import (
    QueryCassandraDatabaseTool,
    GetSchemaCassandraDatabaseTool,
    GetTableDataCassandraDatabaseTool
)
from langchain_community_placeholder.utilities.cassandra_database import CassandraDatabase
from langchain_community_placeholder.agent_toolkit.cassandra_database.toolkit import CassandraDatabaseToolkit
from langchain_community_placeholder.tools.cassandra_database.prompt import QUERY_PATH_PROMPT
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import AIMessage, SystemMessage
from dotenv import load_dotenv
from langchain import hub

# pull in environment variables
load_dotenv()

True

In [2]:
# Create a CassandraDatabase instance
db = CassandraDatabase(contact_points="127.0.0.1", keyspace="killrvideo")

# Create the Cassandra Database tools
query_tool = QueryCassandraDatabaseTool(db=db)
schema_tool = GetSchemaCassandraDatabaseTool(db=db)
data_tool = GetTableDataCassandraDatabaseTool(db=db)

In [3]:
# Test the tools
print("Executing a CQL query:")
query = "SELECT * FROM killrvideo.users LIMIT 5;"
result = query_tool.run({"query": query})
print(result)

print("\nGetting the schema for a keyspace:")
keyspace = "killrvideo"
schema = schema_tool.run({"keyspace": keyspace})
print(schema)

print("\nGetting data from a table:")
keyspace = "killrvideo"
table = "users"
predicate = "userid = 522b1fe2-2e36-4cef-a667-cd4237d08b89"
data = data_tool.run({"keyspace": keyspace, "table": table, "predicate": predicate, "limit": 5})
print(data)

Executing a CQL query:
[{'userid': UUID('bbe83e78-0516-428d-39cb-12706f8c93fd'), 'created_date': None, 'email': None, 'firstname': 'Kenny', 'lastname': '(not'}, {'userid': UUID('bd28a191-21e9-7610-9917-8057dd311189'), 'created_date': None, 'email': None, 'firstname': 'S', 'lastname': 'Shepark'}, {'userid': UUID('9e5442fd-0ba4-39a6-4633-ffc170ca155f'), 'created_date': None, 'email': None, 'firstname': 'Jay', 'lastname': 'Lee'}, {'userid': UUID('67b43ff1-35f9-c64f-7782-6a44b39e7e01'), 'created_date': None, 'email': None, 'firstname': 'P.', 'lastname': '\\P.'}, {'userid': UUID('8b8d8938-9aa1-aa58-ea53-238dd38489a4'), 'created_date': None, 'email': None, 'firstname': 'K.', 'lastname': 'Smith'}]

Getting the schema for a keyspace:
[{'keyspace': 'killrvideo', 'tables': [{'table_name': 'comments_by_user', 'columns': [{'column_name': 'comment', 'type': 'text'}, {'column_name': 'commentid', 'type': 'timeuuid'}, {'column_name': 'userid', 'type': 'uuid'}, {'column_name': 'videoid', 'type': 'uuid'

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
toolkit = CassandraDatabaseToolkit(db=db)

#context = toolkit.get_context()
#tools = toolkit.get_tools()
tools = [schema_tool, data_tool]

input = QUERY_PATH_PROMPT + "\n\nHere is your task: Find all the videos that the user with the email address 'patrick@datastax.com' has uploaded in the killrvideo keyspace. list them in a table format. "

prompt = hub.pull("hwchase17/openai-tools-agent")

# messages = [
#     HumanMessagePromptTemplate.from_template(input),
#     AIMessage(content=QUERY_PATH_PROMPT),
#     MessagesPlaceholder(variable_name="agent_scratchpad"),
# ]

#prompt = ChatPromptTemplate.from_messages(messages)
#print(prompt)

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

print("Available tools:")
for tool in tools:
    print("\t" + tool.name + " - " + tool.description + " - " + str(tool))


Available tools:
	cassandra_db_schema - 
    Input to this tool is a keyspace name, output is a table description of Apache Cassandra tables.
    If the query is not correct, an error message will be returned.
    If an error is returned, report back to the user that the keyspace doesn't exist and stop.
     - db=<langchain_community_placeholder.utilities.cassandra_database.CassandraDatabase object at 0x151ca6a90>
	cassandra_db_data - 
    Tool for getting data from a table in an Apache Cassandra database. 
    Use the WHERE clause to specify the predicate for the query that uses the primary key. A blank predicate will return all rows. Avoid this if possible. 
    Use the limit to specify the number of rows to return. A blank limit will return all rows.
     - db=<langchain_community_placeholder.utilities.cassandra_database.CassandraDatabase object at 0x151ca6a90>


In [5]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input})

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `cassandra_db_schema` with `{'keyspace': 'killrvideo'}`


[{'keyspace': 'killrvideo', 'tables': [{'table_name': 'comments_by_user', 'columns': [{'column_name': 'comment', 'type': 'text'}, {'column_name': 'commentid', 'type': 'timeuuid'}, {'column_name': 'userid', 'type': 'uuid'}, {'column_name': 'videoid', 'type': 'uuid'}], 'indexes': []}, {'table_name': 'comments_by_video', 'columns': [{'column_name': 'comment', 'type': 'text'}, {'column_name': 'commentid', 'type': 'timeuuid'}, {'column_name': 'userid', 'type': 'uuid'}, {'column_name': 'videoid', 'type': 'uuid'}], 'indexes': []}, {'table_name': 'latest_videos', 'columns': [{'column_name': 'added_date', 'type': 'timestamp'}, {'column_name': 'name', 'type': 'text'}, {'column_name': 'preview_image_location', 'type': 'text'}, {'column_name': 'userid', 'type': 'uuid'}, {'column_name': 'videoid', 'type': 'uuid'}, {'column_name': 'yyyymmdd', 'type': 'text'}], 'indexes': []}, {'table_name': '